# Programming Exercise 4: Neural Networks Learning

> In this exercise, you will implement the backpropagation algorithm for neural networks and apply it to the task of hand-written digit recognition.

## 1. Neural Networks

### 1.1 Generate Random Example Data


Loading the dataset:

In [1]:
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Visualizing the data:

In [2]:
SAMPLES = 6
INPUT_LAYER_SIZE = 5
HIDDEN_LAYER_SIZE = 2
OUTPUT_LAYER_SIZE = 2

In [3]:
mat = {'X':np.random.randint(10, size=(SAMPLES, INPUT_LAYER_SIZE))/10 , 'y': np.random.randint(OUTPUT_LAYER_SIZE, size=(SAMPLES,1))}
mat['X'].shape,mat['y'].shape

((6, 5), (6, 1))

In [4]:
mat['X']

array([[0.8, 0.2, 0.2, 0.1, 0.8],
       [0.4, 0.2, 0.7, 0. , 0.6],
       [0.1, 0.2, 0.9, 0.9, 0.5],
       [0.6, 0.4, 0.4, 0.7, 0.2],
       [0.3, 0.6, 0.8, 0.7, 0.5],
       [0.5, 0.8, 0.5, 0.9, 0. ]])

In [5]:
mat['y']

array([[1],
       [1],
       [1],
       [0],
       [0],
       [0]])

### 1.2 Model representation

<img src="neural_network.png">

* $a_i^{(j)}$ = activation of unit $i$ in layer $j$

* $\Theta^{(j)}$ = matrix of weights controlling function mapping from layer $j$ to layer $j+1$. It's dimension is $s_{j+1}\times(s_j+1)$ where $s_j$ is the number of units in layer $j$ and $s_{j+1}$ is the number of units in layer $j+1$.

* $z^{(j+1)} = \Theta^{(j)}a^{(j)}$

* $h_\Theta(x) = a^{(j+1)} = g(z^{(j+1)}) $



Our neural network has:
* $3$ layers: an input layer, a hidden layer and an output layer
* $9$ input layer units (because the images are of size $3 \times 3$
* $3$ units in the second layer
* $2$ output units 
* $\Theta^{(1)}$ and $\Theta^{(2)}$ are provided
    * $\Theta^{(1)}$ has dimension $2 \times 6$
    * $\Theta^{(2)}$ has dimension $2 \times 3$


## $\Theta $

## The random parameters

In [6]:
mat_weights = {'Theta1': np.random.rand(HIDDEN_LAYER_SIZE, INPUT_LAYER_SIZE+1), 'Theta2': np.random.rand(OUTPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE+1)}


In [7]:
print(mat_weights['Theta1'])
print(mat_weights['Theta2'])

[[0.03484898 0.20135391 0.91813286 0.73306505 0.74191071 0.80427777]
 [0.93464201 0.07437255 0.97877467 0.88409762 0.27779808 0.11433137]]
[[0.79496146 0.4314507  0.56066815]
 [0.59378215 0.95916501 0.18731816]]


Unroll parameters:

In [8]:
nn_params = np.hstack((mat_weights['Theta1'].ravel(order='F'), 
                       mat_weights['Theta2'].ravel(order='F')))

In [9]:
nn_params.shape

(18,)

### 1.3 Feedforward and cost function

Cost function without regularization

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m \sum_{k=1}^K [-y_k^{(i)} log((h_\theta(x^{(i)}))_k) - (1-y_k^{(i)})log(1-(h_\theta(x^{(i)}))_k)]$$

In [10]:
def sigmoid(z):
    z = np.array(z)
    return 1 / (1+np.exp(-z))

In [11]:
def nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y):
    
    theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size+1)], newshape=(hidden_layer_size, input_layer_size+1), order='F')
    theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size+1):], newshape=(num_labels, hidden_layer_size+1), order='F')
    
    m = X.shape[0]
    J = 0
    
    K = num_labels
    X = np.hstack((np.ones((m,1)),X)) #add bias unit

    for i in range(m):
        a1 = X[i]
        
        z2 = a1.dot(theta1.T)
        a2 = sigmoid(z2)
        a2 = np.hstack([1, a2]) ##add bias unit
        
        z3 = a2.dot(theta2.T)
        a3 = sigmoid(z3)
        
        h = a3
        
        yk = np.zeros((K,1)) ##y is as K-dimensional vector
        yk[y[i,0]-1, 0] = 1
        
        j = (-yk.T.dot(np.log(h).T) - (1-yk).T.dot(np.log(1-h).T))
        J = J + (j/m)
    return J


In [12]:
J = nn_cost_function(nn_params, INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE, mat['X'], mat['y'])
print(f'Cost at parameters: {J} \n')

Cost at parameters: [1.97080468] 



## 2. Backpropagation

> In this part of the exercise, you will implement the backpropagation algorithm to compute the gradient for the neural network cost function.

### 2.1 Sigmoid gradient

$$g'(z) = \frac{d}{dz}g(z) = g(z)(1-g(z))$$ 

where $g(z) = sigmoid(z) = \frac{1}{1+e^{-z}}$ 

In [13]:
def sigmoid_gradient(z):
    return sigmoid(z) * (1-sigmoid(z))

### 2.2 Random initialization
> When training neural networks, it is important to randomly initialize the pa- rameters for symmetry breaking. One effective strategy for random initialization is to randomly select values for $\Theta^{(l)}$ uniformly in the range $[−\epsilon_{init}, \epsilon_{init}]$. You should use $\epsilon_{init} = 0.12$.

In [14]:
epsilon_init = 0.12
initial_theta1 = np.random.uniform(low=-epsilon_init, high=epsilon_init, size=(HIDDEN_LAYER_SIZE, INPUT_LAYER_SIZE+1))
initial_theta2 = np.random.uniform(low=-epsilon_init, high=epsilon_init, size=(OUTPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE+1))

### 2.3 Backpropagation

> Given a training example $(x^{(t)},y^{(t)})$, we will first run a "forward pass" to compute all the activations throughout the network, including the output value of the hypothesis $h_\Theta(x)$. Then, for each node $j$ in layer $l$, we would like to compute an "error term" $\delta_j^{(l)}$ that measures how much that node was "responsible" for any errors in our output.

<img src="backpropagation.png">

Algorithm:

1) Set the input layer's values $(a^{(1)})$ to the t-th training example $x^{(t)}$.
Perform a feedforward pass, computing the activations ($z^{(2)}, a^{(2)}, z^{(3)}, a^{(3)}$) for layers 2 and 3.

2) For each output unit $k$ in layer $3$ (the output layer), set 

$$ \delta_k^{(3)} = (a_k^{(3)} - y_k)$$

3) For the hidden layer $l = 2$, set

$$ \delta_k^{(2)} = (\Theta^{(2)})^T \delta^{(3)} .* g'(z^{(2)})$$ 

4) Accumulate the gradient from this example using the following formula. Note that you should skip or remove $\delta_0^{(2)}$

$$ \Delta^{(l)} = \Delta^{(l)} + \delta^{(l+1)}(a^{(l)})^T$$

5) Obtain the (unregularized) gradient for the neural network cost func-
tion by dividing the accumulated gradients by $\frac{1}{m}$:

$$ D_{ij}^{(l)} = \frac{1}{m}\Delta_{ij}^{(l)}$$

In [15]:
def nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r):
    
    theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size+1)], newshape=(hidden_layer_size, input_layer_size+1), order='F')
    theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size+1):], newshape=(num_labels, hidden_layer_size+1), order='F')
    
    m = X.shape[0]
    J = 0
    
    K = num_labels
    X = np.hstack((np.ones((m,1)),X)) #add bias unit

    capital_delta1 = np.zeros(theta1.shape)
    capital_delta2 = np.zeros(theta2.shape)
    
    for i in range(m):
        a1 = X[i]
        
        z2 = a1.dot(theta1.T)
        a2 = sigmoid(z2)
        a2 = np.hstack([1, a2]) ##add bias unit
        
        z3 = a2.dot(theta2.T)
        a3 = sigmoid(z3)
        
        h = a3
        
        yk = np.zeros((K,1)) ##y is as K-dimensional vector
        yk[y[i,0]-1, 0] = 1
        
        j = (-yk.T.dot(np.log(h).T) - (1-yk).T.dot(np.log(1-h).T)) ##sum of K
        J = J + (j/m) #sum of i
        
        delta3 = a3 - yk.T
        
        z2 = np.hstack([1, z2])
        delta2 = theta2.T.dot(delta3.T) * (sigmoid_gradient(z2).reshape(-1,1))
        
        capital_delta1 = capital_delta1 + (delta2[1:,:].dot(a1.reshape(1,-1)))
        capital_delta2 = capital_delta2 + (delta3.T.dot(a2.reshape(1,-1)))
        
    sum1 = np.sum(np.sum(theta1[:, 1:] ** 2))
    sum2 = np.sum(np.sum(theta2[:, 1:] ** 2))
    J = J + (lambda_r / (2*m)) * (sum1 + sum2)
    
    theta1_grad = (1/m) * (capital_delta1 + lambda_r * theta1) #with regularization
    theta1_grad[:,0] = ((1/m) * capital_delta1)[:,0]
    
    theta2_grad = (1/m) * (capital_delta2 + lambda_r * theta2) #with regularization
    theta2_grad[:,0] = ((1/m) * capital_delta2)[:,0]
    
    grad = np.hstack((theta1_grad.ravel(order='F'), theta2_grad.ravel(order='F')))
    return J, grad

### 2.4 Gradient Checking

> In your neural network, you are minimizing the cost function $J(\Theta)$. To perform gradient checking on your parameters, you can imagine “unrolling” the parameters $\Theta^{(1)}, \Theta^{(2)}$ into a long vector $\theta$. By doing so, you can think of the cost function being $J(\theta)$ instead and use the following gradient checking procedure. Suppose you have a function $f_i(\theta)$ that purportedly computes $\frac{\partial}{\partial \theta_i} J(\theta)$; you’d like to check if $f_i$ is outputting correct derivative values. If $\theta^{(i+)}$ is the same $\theta$, except its i-th element has been incremented by $\epsilon$. You can now numerically verify $f_i(\theta)$’s correctness by checking, for each $i$, that: $f_i(\theta) 	\approx \frac{J(\theta^{(i+)} - J(\theta^{(i-)}}{2\epsilon}$. The degree to which these two values should approximate each other will depend on the details of $J$. But assuming $\epsilon = 10^{−4}$, you’ll usually find that the left- and right-hand sides of the above will agree to at least 4 significant digits (and often many more).

In [16]:
def compute_numerical_gradient(theta, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r):
    e = 0.0001
    num_grad = np.zeros(theta.shape)
    perturb = np.zeros(theta.shape)
    for p in range(len(theta)):
        perturb[p] = e
        loss1, _ = nn_cost_function(theta-perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
        loss2, _ = nn_cost_function(theta+perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
        num_grad[p] = (loss2-loss1)/(2*e)
        perturb[p] = 0
    return num_grad

In [17]:
def debug_initialize_weights(fan_out, fan_in):
    W = np.zeros((fan_out, 1+fan_in))
    W = np.reshape(range(len(W.ravel(order='F'))), W.shape)/10
    return W

In [18]:
def check_nn_gradients(lambda_r=0):
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    
    theta1 = debug_initialize_weights(hidden_layer_size, input_layer_size)
    theta2 = debug_initialize_weights(num_labels, hidden_layer_size)
    
    X = debug_initialize_weights(m, input_layer_size-1)
    y = 1 + np.mod(range(m), num_labels).reshape(-1, 1)
    
    nn_params = np.hstack((theta1.ravel(order='F'), theta2.ravel(order='F')))
    
    cost, grad = nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
    num_grad = compute_numerical_gradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_r)
    
    print('The columns should be very similar...')
    for i, j in zip(num_grad, grad):
        print(i,j)
        
    diff = np.linalg.norm(num_grad-grad)/np.linalg.norm(num_grad+grad)
    if diff < 0.000000010:
        print('\nBackpropagation is correct')
    else:
        print('\nBackpropagation is incorrect')

In [19]:
check_nn_gradients()

The columns should be very similar...
0.3508102791105472 0.35081027824835614
0.2353522188158763 0.23535221882914456
0.1452979107607888 0.14529791114175525
0.09315199914539107 0.09315199923194291
0.06032069570061083 0.06032069599693199
0.1937398015439129 0.19373980185067552
0.08735383096869498 0.0873538307836033
0.029773973508895324 0.029773974437035562
0.010590364460938417 0.010590364574492521
0.004047666530837546 0.00404766588020231
0.2288208313494522 0.22882082967551112
0.11088905266909421 0.11088905266651777
0.04430376599806607 0.04430376555121108
0.019905564396793807 0.01990556449768681
0.010079735703882875 0.010079735479895512
0.26390185717595216 0.2639018575003467
0.1344242739431678 0.1344242745494322
0.058833556648707486 0.0588335566653866
0.029220764217186 0.029220764420881108
0.01611180537430812 0.01611180507958871
0.39622224609736634 0.3962222462654107
0.5887339968513317 0.588733996850061
0.7994663673738245 0.7994663670394307
0.24984228291557997 0.2498422829571028
0.349065613

In [20]:
lambda_r = 3
check_nn_gradients(lambda_r)
J, grad = nn_cost_function(nn_params, INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE, mat['X'], mat['y'], lambda_r)
print('Cost at parameters (loaded from ex4weights): {0} \n(this value should be about 0.576051)'.format(J))

The columns should be very similar...
0.35081027911942897 0.35081027824835614
0.2353522188158763 0.23535221882914456
0.145297910751907 0.14529791114175525
0.09315199914539107 0.09315199923194291
0.06032069569172904 0.06032069599693199
0.2537398015434178 0.2537398018506755
0.38735383096621945 0.3873538307836033
0.5697739734955576 0.5697739744370356
0.7905903644456203 0.7905903645744927
1.0240476665224207 1.0240476658802022
0.348820831348462 0.34882082967551115
0.47088905267500536 0.47088905266651776
0.6443037660019968 0.6443037655512112
0.8599055643898623 0.8599055644976867
1.0900797357216163 1.0900797354798957
0.44390185717446684 0.4439018575003467
0.5544242739397021 0.5544242745494321
0.7188335566432613 0.7188335566653867
0.9292207642097594 0.9292207644208812
1.1561118053826647 1.1561118050795887
0.39622224608848455 0.3962222462654107
0.5887339968602134 0.588733996850061
0.7994663673827063 0.7994663670394307
0.30984228292396665 0.3098422829571028
0.7690656134862195 0.769065613486773
1

### 2.5 Learning Parameters - Training the Neural Network

In [21]:
import scipy.optimize as opt
lambda_r = 1
opt_results = opt.minimize(nn_cost_function, nn_params, args=(INPUT_LAYER_SIZE, 
                                                              HIDDEN_LAYER_SIZE, 
                                                              OUTPUT_LAYER_SIZE, 
                                                              mat['X'], 
                                                              mat['y'], 
                                                              lambda_r), 
                            method='L-BFGS-B', jac=True, options={'maxiter':50})

In [22]:
opt_results

      fun: array([1.38629436])
 hess_inv: <18x18 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.80792055e-11, -1.98046559e-12, -1.74845869e-07,  4.32900245e-07,
       -6.96495258e-07,  1.57157402e-06, -2.72298674e-08,  3.24587567e-07,
       -7.05634771e-07,  1.72611104e-06,  5.29772446e-07, -9.25611687e-07,
       -3.40538937e-06, -4.62359157e-06, -9.42962992e-06,  5.26887127e-06,
       -3.12267523e-06, -1.20305879e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 15
      nit: 13
   status: 0
  success: True
        x: array([-9.47611046e-02,  9.10817822e-01,  2.32436356e-07,  2.85962323e-06,
        1.11985910e-05,  1.25761058e-05, -1.44477872e-06,  1.68531582e-06,
        1.24252256e-05,  1.37655600e-05, -1.21988370e-05, -8.70032333e-06,
        1.74758478e-05, -5.46224432e-05, -5.23825102e-05,  5.03645652e-05,
       -8.61801746e-06,  1.70197716e-05])

In [23]:
theta1 = np.reshape(opt_results['x'][:HIDDEN_LAYER_SIZE * (INPUT_LAYER_SIZE+1)], newshape=(HIDDEN_LAYER_SIZE, INPUT_LAYER_SIZE+1), order='F')
theta2 = np.reshape(opt_results['x'][HIDDEN_LAYER_SIZE * (INPUT_LAYER_SIZE+1):], newshape=(OUTPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE+1), order='F')


In [24]:
def predict_nn(theta1, theta2, X):
    m, n = X.shape
    a1 = np.hstack((np.ones((m,1)),X)) #with a0
    
    z2 = a1.dot(theta1.T)
    a2 = sigmoid(z2)
    
    z3 = np.hstack((np.ones((m,1)),a2)).dot(theta2.T) #with a0
    a3 = sigmoid(z3)
    h = np.argmax(a3, axis=1)+1 #get label with largest h(x)
    
    return h

In [25]:
y_pred = predict_nn(theta1, theta2, mat['X'])
accuracy = np.mean(y_pred == mat['y'].T)
f'Train accuracy: {accuracy * 100}'

'Train accuracy: 50.0'

#### 2.5.1 Similar Code using Scikit-Learn:

In [26]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(HIDDEN_LAYER_SIZE,), activation='logistic', solver='lbfgs', alpha=1, max_iter=50)
nn.fit(mat['X'], mat['y'].T[0])

MLPClassifier(activation='logistic', alpha=1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(2,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=50,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [27]:
nn.score(mat['X'], mat['y'].T[0])

0.5

## 3. Visualizing the hidden layer

In [28]:
theta1

array([[-9.47611046e-02,  2.32436356e-07,  1.11985910e-05,
        -1.44477872e-06,  1.24252256e-05, -1.21988370e-05],
       [ 9.10817822e-01,  2.85962323e-06,  1.25761058e-05,
         1.68531582e-06,  1.37655600e-05, -8.70032333e-06]])